In [ ]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [ ]:
import boto3
import pandas as pd
import numpy as np
from IPython.display import display

"""
ec2 = boto3.client('ec2', region_name='us-west-2')
response = ec2.describe_instances()
print(response)oro
"""

def highlight_reserved(df_row):
    if df_row.name[0] in ['r4.8xlarge','m5.12xlarge']:
        return ['background-color: yellow; border: 1px  black solid !important; color: black !important']*2
    else:
        return ['border: 1px  black solid !important; color: black !important']*2
    
emr = boto3.client('emr' , region_name='us-west-2')

page_iterator = emr.get_paginator('list_clusters').paginate(
    ClusterStates=['RUNNING','WAITING']
)

clusters_active = []
for page in page_iterator:
    clusters_active.extend(page['Clusters'])

print("Total Active Clusters:", len(clusters_active))

emr_df = pd.DataFrame()
for cluster in clusters_active:
    cluster_id = cluster['Id']
    instance_groups = emr.list_instance_groups(ClusterId=cluster_id)['InstanceGroups']
    instance_groups_df = pd.DataFrame(instance_groups)
    instance_groups_df['Cluster_Name'] = cluster['Name']
    instance_groups_df['Cluster_Id'] = cluster['Id']
    emr_df = emr_df.append(instance_groups_df[['Cluster_Name','Cluster_Id','InstanceType', 'InstanceGroupType', 'Market', 'RunningInstanceCount' ]], ignore_index=True)

instance_total = emr_df['RunningInstanceCount'].sum()
print('Total Running instances count within EMR: {}'.format(instance_total))
#instance_type_cnt = instance_type_grp['RunningInstanceCount'].agg({'Count': np.sum})
#instance_type_cnt = instance_type_grp.agg({'RunningInstanceCount' : 'sum', 'Cluster_Instance' : lambda x: ', '.join(x)})
instance_grp = emr_df.groupby(['InstanceType','Market'])
#instance_grp_cnt = instance_grp.agg({'RunningInstanceCount' : 'sum'})
instance_grp_cnt = instance_grp.agg({'RunningInstanceCount' : 'sum', 'Cluster_Name' : lambda x: ', '.join(x)}).reindex(['RunningInstanceCount','Cluster_Name'], axis=1)
instance_grp_cnt = instance_grp_cnt[instance_grp_cnt['RunningInstanceCount'] != 0]
print("Count of InstanceType accross EMR")
#instance_grp_cnt = instance_grp_cnt.style.apply(highlight_reserved, axis=1)
display(instance_grp_cnt)

instance_emr_cnt = emr_df.groupby(['InstanceType','Cluster_Name','Market']).agg({'RunningInstanceCount' : 'sum'})
instance_emr_cnt = instance_emr_cnt[instance_emr_cnt['RunningInstanceCount'] != 0]
print("Distribution of InstanceType per EMR")
display(instance_emr_cnt)

emr_instance_cnt = emr_df.groupby(['Cluster_Name','InstanceType','Market']).agg({'RunningInstanceCount' : 'sum'})
emr_instance_cnt = emr_instance_cnt[emr_instance_cnt['RunningInstanceCount'] != 0]
print("Distribution of InstanceType on EMR")
display(emr_instance_cnt)

print("Distribution of Instances per Cluster")
emr_node_per_cluster = emr_df.groupby(['Cluster_Name']).agg({'RunningInstanceCount' : 'sum'})
display(emr_node_per_cluster)
